In [4]:
import numpy as np
from numba import njit

In [15]:
# Model parameters
median = 1.75/1000
γ = 0.018
τ = median * γ
δ = 0.01
η = 0.032
ξ_m = 0.00256

μ_2 = 1.
ρ = 0.5
σ_2 = np.sqrt(0.21**2*2*ρ/μ_2) # Match moments, using 100 year's std

Equation:

$$
0 = \max_{e} \min_{h_2} - \delta \phi(r, z_2) + \delta \eta \log e - \tau z_2 e - \frac{\partial \phi}{\partial r}(r, z_2) e + \xi_m\frac{(h_2)^2}{2} + \left[\frac{\partial \phi}{\partial z_2}(r, z_2)\right]\left[-\rho (z_2 -\mu_2) + \sqrt{z_2} \sigma_2 h_2\right] + \left[\frac{\partial^2 \phi}{\partial(z_2)^2}(r, z_2)\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)
$$

FOC of $h_2$ gives : 

$$
h_2^* = -\frac{\left[\frac{\partial \phi}{\partial z_2}(r, z_2)\right]\sqrt{z_2}\sigma_2}{\xi_m}
$$

FOC of $e$ gives :

$$
e^* = \frac{\delta \eta}{\tau z_2 + \frac{\partial \phi}{\partial r}(r, z_2)}
$$

Substituting terms we have:

$$
0 = - \delta \phi(r, z_2) + \delta \eta \log e^* - \tau z_2 e^* - \frac{\partial \phi}{\partial r}(r, z_2) e^* - \frac{\xi_m}{2}\left[\frac{\partial \phi}{\partial z_2}(r, z_2)\right]^2z_2|\sigma_2|^2 + \left[\frac{\partial \phi}{\partial z_2}(r, z_2)\right]\left[-\rho (z_2 -\mu_2)\right] + \left[\frac{\partial^2 \phi}{\partial(z_2)^2}(r, z_2)\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)
$$

False transient method:

$$
\frac{{\color{red}{\phi_{i+1}(r, z_2)}}-{\color{blue}{\phi_{i}(r, z_2)}}}{\epsilon} = - \delta {\color{red}{\phi_{i+1}(r, z_2)}} + \delta \eta \log \color{blue}{e^*} - \tau z_2 \color{blue}{e^*} - \color{red}{\frac{\partial \phi_{i+1}}{\partial r}(r, z_2)} \color{blue}{e^*} - \frac{\xi_m}{2}\left[\color{blue}{\frac{\partial \phi_i}{\partial z_2}(r, z_2)}\right]^2z_2|\sigma_2|^2 + \left[\color{red}{\frac{\partial \phi_{i+1}}{\partial z_2}(r, z_2)}\right]\left[-\rho (z_2 -\mu_2)\right] + \left[\color{red}{\frac{\partial^2 \phi_{i+1}}{\partial(z_2)^2}(r, z_2)}\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)
$$

where

\begin{align*}
\color{red}{\frac{\partial \phi_{i+1}}{\partial r}(r, z_2)} & \approx \frac{\phi_{i+1}(r+\Delta r, z_2) - \phi_{i+1}(r, z_2)}{\Delta r} \mathbb{1}\{e^*<0\} + \frac{\phi_{i+1}(r, z_2) - \phi_{i+1}(r- \Delta r, z_2)}{\Delta r} \mathbb{1}\{e^*>0\}\\
\color{red}{\frac{\partial \phi_{i+1}}{\partial z_2}(r, z_2)} & \approx \frac{\phi_{i+1}(r, z_2+\Delta z) - \phi_{i+1}(r, z_2)}{\Delta z} \mathbb{1}\{z_2<\mu_2\} + \frac{\phi_{i+1}(r, z_2) - \phi_{i+1}(r, z_2- \Delta z)}{\Delta z} \mathbb{1}\{z_2>\mu_2\}\\
\color{red}{\frac{\partial^2 \phi_{i+1}}{\partial z_2^2}(r, z_2)} & \approx \frac{\phi_{i+1}(r, z_2 + \Delta z) - 2\phi_{i+1}(r, z_2) + \phi_{i+1}(r, z_2-\Delta z)}{(\Delta z)^2}
\end{align*}

Notice that on the boundary points, we 1) use one-sided difference to estimate the first-order derivative and 2) assume the second-order derivative is the same to the second-order derivative of the point next to it.

In [ ]:
@njit(parallel=True, cache=True)
def solver(ϕ_grid, r_grid, z_grid, ϵ, τ, δ, η, ξ_m, μ_2, σ_2, ρ):
    n_r = len(r_grid)
    n_z = len(z_grid)
    Δ_r = r_grid[1] - r_grid[0]
    Δ_z = z_grid[1] - z_grid[0]
    LHS = np.zeros((n_r*n_z, n_r*n_z))
    RHS = np.zeros(n_r*n_z)
    for j in range(n_z):
        for i in range(n_r):
            # Get index
            idx = j*n_r + i
            idx_rp1 = idx + 1
            idx_rm1 = idx - 1
            idx_zp1 = (j+1)*n_r + i
            idx_zp2 = (j+2)*n_r + i
            idx_zm1 = (j-1)*n_r + i
            idx_zm2 = (j-2)*n_r + i
            ϕ = ϕ_grid[idx]
            z = z_grid[j]
            LHS[idx, idx] = -δ - 1./ϵ
            # Compute values in blue
            if i == 0:
                dϕdr = (ϕ_grid[idx_rp1]-ϕ_grid[idx])/Δ_r
                e = δ*η/(τ*z+dϕdr)
                LHS[idx, idx] += e/Δ_r
                LHS[idx, idx_rp1] += -e/Δ_r
            elif i == n_r-1:
                dϕdr = (ϕ_grid[idx]-ϕ_grid[idx_rm1])/Δ_r
                e = δ*η/(τ*z+dϕdr)
                LHS[idx, idx] += -e/Δ_r
                LHS[idx, idx_rm1] += e/Δ_r
            else:
                dϕdr = (ϕ_grid[idx_rp1]-ϕ_grid[idx_rm1])/(2*Δ_r)
                e = δ*η/(τ*z+dϕdr)
                LHS[idx, idx] += e/Δ_r*(e<0) - e/Δ_r*(e>0)
                LHS[idx, idx_rp1] += -e/Δ_r*(e<0)
                LHS[idx, idx_rm1] += e/Δ_r*(e>0)
            temp_1 = - ρ*(z-μ_2)
            temp_2 = z*σ_2**2/2
            if j == 0:
                dϕdz = (ϕ_grid[idx_zp1]-ϕ_grid[idx])/Δ_z
                LHS[idx, idx] += -temp_1/Δ_z + temp_2/(Δ_z**2)  
                LHS[idx, idx_zp1] += temp_1/Δ_z - temp_2*2/(Δ_z**2)
                LHS[idx, idx_zp2] += temp_2/(Δ_z**2)
            elif j == n_z-1:
                dϕdz = (ϕ_grid[idx]-ϕ_grid[idx_zm1])/Δ_z
                LHS[idx, idx] += temp_1/Δ_z + temp_2/(Δ_z**2)
                LHS[idx, idx_zm1] += -temp_1/Δ_z - temp_2*2/(Δ_z**2)
                LHS[idx, idx_zm2] += temp_2/(Δ_z**2)            
            else:
                dϕdz = (ϕ_grid[idx_zp1]-ϕ_grid[idx_zm1])/(2*Δ_z)
                LHS[idx, idx] += temp_1/Δ_z*(-1.*(z<μ_2)+(z>μ_2)) - temp_2*2/(Δ_z**2)
                LHS[idx, idx_zp1] += temp_1/Δ_z*(z<μ_2) + temp_2/(Δ_z**2)
                LHS[idx, idx_zm1] += -temp_1/Δ_z*(z>μ_2) + temp_2/(Δ_z**2)
            RHS[idx] = -(1./ϵ*ϕ + δ*η*np.log(e) - τ*z*e - ξ_m/2*dϕdz**2*z*σ_2**2)
    ϕ = np.linalg.solve(LHS, RHS)
    return ϕ


@njit
def false_transient(ϕ_grid, r_grid, z_grid, ϵ, τ, δ, η, ξ_m, μ_2, σ_2, ρ, max_iter=10_000, tol=1e-9):
    n_r = len(r_grid)
    n_z = len(z_grid)
    error = 1.
    count = 0
    while error > tol and count < max_iter:
        ϕ_grid_old = ϕ_grid.copy()
        ϕ_grid = solver(ϕ_grid, r_grid, z_grid, ϵ, τ, δ, η, ξ_m, μ_2, σ_2, ρ)
        error = np.max(np.abs(ϕ_grid_old-ϕ_grid))/ϵ
        count += 1
        print('Iteration:', count, ', error:', error)
    return ϕ_grid

In [ ]:
ϵ = 0.5
r_max = 9000.
z_max = 4.
n_r = 200
n_z = 20
r_grid = np.linspace(0, r_max, n_r)
z_grid = np.linspace(1e-5, z_max, n_z)
ϕ_grid = np.zeros(n_r*n_z) # initial guess

In [ ]:
ϕ_grid = false_transient(ϕ_grid, r_grid, z_grid, ϵ, τ, δ, η, ξ_m, μ_2, σ_2, ρ, max_iter=10_000, tol=1e-7)